In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import cv2
import datetime
import random


2024-08-16 01:39:30.398060: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-16 01:39:30.527588: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-16 01:39:30.527713: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-16 01:39:30.551033: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-16 01:39:30.593288: I tensorflow/core/platform/cpu_feature_guar

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import load_model
# from keras.applications.vgg16 import VGG16
# from keras.applications.vgg16 import preprocess_input
# from keras.applications.efficientnet import EfficientNetB7
# from keras.applications.efficientnet import preprocess_input
from keras.applications.resnet_v2 import ResNet50V2
from keras.applications.resnet_v2 import preprocess_input

In [3]:
# CPU settigs

cpus = tf.config.list_physical_devices('CPU')

try:
    tf.config.set_visible_devices(cpus[0], 'CPU')
    print("Switching to CPU")
except:
    pass

# tf.debugging.set_log_device_placement(True)

Switching to CPU


2024-08-16 01:39:32.197339: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-08-16 01:39:32.268006: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-08-16 01:39:32.268346: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [4]:
# # GPU settings



# gpus = tf.config.list_physical_devices('GPU')



# # Setting gpu[0] as main device
# if gpus:
#   try:
#     tf.config.set_visible_devices(gpus[0], 'GPU')
#     logical_gpus = tf.config.list_logical_devices('GPU')
#     print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
#   except RuntimeError as e:
#     # Visible devices must be set before GPUs have been initialized
#     print(e)


# # Setting memory growth
# try:
#   tf.config.experimental.set_memory_growth(gpus[0], True)
# except:
#   # Invalid device or cannot modify virtual devices once initialized.
#   pass



# # tf.debugging.set_log_device_placement(True)


# os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'
# print(os.getenv('TF_GPU_ALLOCATOR'))

In [5]:
#Initialization of selective search
ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()

In [6]:
#Path to images and folder with bounding boxes of all airplane locations in an image for all images
images_path = "/home/abhinav/Documents/Work/2 Hobby_projects/Datasets/Aiplanes_dataset_for_rcnn/Images"
label_path = "/home/abhinav/Documents/Work/2 Hobby_projects/Datasets/Aiplanes_dataset_for_rcnn/Airplanes_Annotations"

model_dir = "/home/abhinav/Documents/Work/2 Hobby_projects/Models"

In [7]:
#there are 700 images with name starting with airplane, and 33 with numbers
#to hold the image name and their bb of all airplane locations
img_bb = {}

In [8]:
for label in os.listdir(label_path):                        # It gives each element of label_path directory to here (label)
    if label.split("_")[0] == "airplane":                   # We are only considering these images
        try:
            file = pd.read_csv(label_path+"/"+label)        # Reading each csv file containing bb of each image
            # print("label: ", label)
            # print("len of file: ", len(file))
            labels_for_image = []                           # To store the bb in integer format from csv
            for row in file.iterrows():
                # for k in row:
                #     print(row[k][0].split(" "))
                bb_each = list(row[1])[0].split(" ")
                for j in range(len(bb_each)):
                    bb_each[j] = eval(bb_each[j])

                labels_for_image.append(bb_each)            # Labels_for_image is the list with all bb of current image
            img_bb[label.split(".")[0]] = labels_for_image  # We choose file name without jpg as key and labels_for_image as their value
        except Exception as e:
            print(e)
            print("Error in :", label)

In [9]:
# img_bb is a dictionary that now contains image name as key and their true bb of each airplane as a list of lists as the value of dictionary
print("Total images for training: ", len(img_bb))

Total images for training:  700


In [10]:
# To check how data is stored
# c = 0
# for i in img_bb:
#     if c == 4:
#         break
#     print("Image name: ", i)
#     print("Number of labels: ", len(img_bb[i]))
#     print("Labels: ", img_bb[i])
#     c += 1

In [11]:
# To calculate the area of intersection
def gt(bb1, bb2):
    assert bb1['x1'] < bb1['x2']
    assert bb1['y1'] < bb1['y2']
    assert bb2['x1'] < bb2['x2']
    assert bb2['y1'] < bb2['y2']
    x_left = max(bb1['x1'], bb2['x1'])
    y_top = max(bb1['y1'], bb2['y1'])
    x_right = min(bb1['x2'], bb2['x2'])
    y_bottom = min(bb1['y2'], bb2['y2'])
    if x_right < x_left or y_bottom < y_top:
        return 0.0
    intersection_area = (x_right - x_left) * (y_bottom - y_top)
    bb1_area = (bb1['x2'] - bb1['x1']) * (bb1['y2'] - bb1['y1'])
    bb2_area = (bb2['x2'] - bb2['x1']) * (bb2['y2'] - bb2['y1'])
    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
    assert iou >= 0.0
    assert iou <= 1.0
    return iou

In [12]:
# '''
# 1   Reading each image from folder
# 2   we consider only images starting with name airplane
# 3   Performing selective search
# 4   Taking each region proposal of ss, Each region propasal of ss is given as an array as [x, y, w, h]
#         where (x, y) are top left corner coordinates of bb and w, h are width and height
# 5   Here we do not add width and height as the label data cause:
# 6   we downloaded has bb of the format [x1, y1, x2, y2]
#         (x1, y1): top left corner coordinates
#         (x2, y2): bottom right corner coordinates
# 7   Here iou is greater than 0.7, if we haven't added 30 of this class to train_images:
#         add the resized image of this proposed bounding box by selective search to train_images



# Steps involved in displaying an image in opencv:
#     cv2.imshow("imout: ", imout)
#     cv2.waitKey(0),
#     cv2.destroyAllWindows()


# '''

In [13]:
pos_threshold = 0.83
neg_threshold = 0.70
no_images_to_be_used_for_extraction = 500
no_of_positive_images_each = 20
no_of_neg_images_each = 45

In [14]:
# These lines can display the image you want

# i = cv2.imread("/home/abhinav/Documents/Work/2 Hobby_projects/Datasets/Aiplanes_dataset_for_rcnn/Images/airplane_343.jpg")
# i = cv2.imread("/home/abhinav/Documents/Work/2 Hobby_projects/Datasets/Aiplanes_dataset_for_rcnn/Images/42845.jpg")

# cv2.imshow("image", i)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [15]:
# Displaying images of x_new_images resized images:

# for index, i in enumerate(x_new_images):
#     if index > 4:
#         break
#     cv2.imshow("image: " + str(y_new_labels[index]), i)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()

In [16]:
#we pass each image for selective search and calculate iou for each bb output of ss out of 2000 proposals

x_new_images, y_new_labels = [], []
no_images = 0

total_images_extracted = 0
# s = random.sample(range(0, 700), no_images_to_be_used_for_extraction)
s = [x for x in range(int(no_images_to_be_used_for_extraction/5))]
print("random sample list: ", s)
for img in os.listdir(images_path):             # 1
    try:
        if img.split("_")[0] == "airplane":
            if int(img.split("_")[1].split(".")[0]) in s:                              # 2

                image = cv2.imread(os.path.join(images_path, img))
                # print("image original shape: ", image.shape)
                ss.setBaseImage(image)                                       # 3
                ss.switchToSelectiveSearchFast(base_k=10)
                ssresults = ss.process()
                imout = image.copy()
                # print("imout shape: ", imout.shape)
                # cv2.imshow("imout: ", imout)
                # cv2.waitKey(0),
                # cv2.destroyAllWindows()


                # ele1n = os.path.join(images_path, img)

                # In openCV, consider a square window facing us, 
                # its left side will be y axis and top side will be x axis
                # top left corner is considered as (0, 0)

                iou_07 = 0
                iou_03 = 0
                m = random.sample(range(len(ssresults)), len(ssresults))
                for j in m:
                    i = ssresults[j]
                    x1, y1, w, h = i                                          # 4
                    bb2 = {"x1":x1, "y1":y1, "x2":x1+w, "y2":y1+h}            
                    img = img.split(".")[0]
                    for j in img_bb[img]:                                     # For each true bb in img_bb, we calculate iou for the bb region proposed by ss
                        x, y, xn, yn = j                                      # 5
                        bb1 = {"x1":x, "y1":y, "x2":xn, "y2":yn}              # 6
                        iou = gt(bb1, bb2)                                    # Calculating iou
                        if iou > pos_threshold and iou_07 < no_of_positive_images_each:                         # 7
                            # print("iou07: bb: x1, y1, w, h, x1+w, y1+h:", x1, y1, w, h, x1+w, y1+h)
                            timage = imout[y1:y1+h,x1:x1+w]
                            # print("timage shape: ", timage.shape)
                            resized = cv2.resize(timage, (224, 224), interpolation=cv2.INTER_AREA)
                            # print("resized timage shape: ", resized.shape)
                            x_new_images.append(resized)
                            y_new_labels.append(1)
                            iou_07 += 1
                            # cv2.imshow("resized: ", resized)
                            # cv2.waitKey(0)
                            # cv2.destroyAllWindows()
                            # cv2.rectangle(image, (bb1["x1"], bb1["y1"]), (bb1["x2"], bb1["y2"]), (255, 0, 0), 2)
                            # cv2.rectangle(image, (bb2["x1"], bb2["y1"]), (bb2["x2"], bb2["y2"]), (0, 255, 0), 1)
                        elif iou < neg_threshold and iou_03 < no_of_neg_images_each:
                            timage = imout[y1:y1+h, x1:x1+h]
                            resized = cv2.resize(timage, (224, 224), interpolation=cv2.INTER_AREA)
                            x_new_images.append(resized)
                            y_new_labels.append(0)
                            iou_03 += 1
                # cv2.imshow(img, image)
                # cv2.waitKey(0)
                # cv2.destroyAllWindows()
                # print("for image: ", img, " there are _ iou07 and _ iou03: ", iou_07, iou_03)
                total_images_extracted += iou_03 + iou_07
            no_images += 1
    except Exception as e:
        print(e)
        print("Error in image: ", img)
        continue

    

x_new = np.array(x_new_images)
y_new = np.array(y_new_labels)



arr = np.array(y_new)
C = tf.constant(2)

encoded_y = tf.one_hot(arr, C, axis=-1)


del x_new_images, y_new_labels, s

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=3,
    width_shift_range=3,
    height_shift_range=3,
    shear_range=1,
    zoom_range=5,
    horizontal_flip=True,
    vertical_flip=True,
    rescale=1./255,
    validation_split=0.35,
)


train_images = train_datagen.flow(
    x=x_new,
    y=encoded_y,
    batch_size=16,
    shuffle=True
)

# valid_datagen = ImageDataGenerator(
#     rescale=1.0/255.0
# )

# valid_images = valid_datagen.flow(
#     x=,
#     y=y2,
#     batch_size=16,
#     shuffle=True
# )
# del x2, y2
# train_images3 = train_datagen.flow(
#     x=x3,
#     y=y3,
#     batch_size=64,
#     shuffle=True
# )


del x_new, encoded_y

random sample list:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]


2024-08-16 01:40:00.496964: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-08-16 01:40:00.497150: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-08-16 01:40:00.497215: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [17]:
# Now we have in total 1246 images of positive and negative samples mix to fine tune the next VGGNet model

print("total images extracted and resized: ", total_images_extracted)

total images extracted and resized:  4848


In [18]:
%load_ext tensorboard

In [19]:
model = load_model(os.path.join(model_dir, "Mynet1"))
# model = VGG16(input_shape=(224, 224, 3), weights='imagenet', include_top=True)
# model = EfficientNetB7(input_shape=(600, 600, 3), weights='imagenet', include_top=True)
# model = ResNet50V2(input_shape=(224, 224, 3), weights="imagenet", include_top=True)

for layer in model.layers:
    layer.trainable = False

i = 0
for layer in model.layers:
    i += 1
    print("layer:", str(i), layer)

layer: 1 <keras.src.layers.convolutional.conv2d.Conv2D object at 0x7131ac53ba30>
layer: 2 <keras.src.layers.convolutional.conv2d.Conv2D object at 0x7131ac66add0>
layer: 3 <keras.src.layers.pooling.max_pooling2d.MaxPooling2D object at 0x7131ac66b6d0>
layer: 4 <keras.src.layers.convolutional.conv2d.Conv2D object at 0x7131ac4e0340>
layer: 5 <keras.src.layers.convolutional.conv2d.Conv2D object at 0x7131ac51db70>
layer: 6 <keras.src.layers.pooling.max_pooling2d.MaxPooling2D object at 0x7131ac51e830>
layer: 7 <keras.src.layers.convolutional.conv2d.Conv2D object at 0x7131ac51d9f0>
layer: 8 <keras.src.layers.pooling.max_pooling2d.MaxPooling2D object at 0x7131ac51e0e0>
layer: 9 <keras.src.layers.reshaping.flatten.Flatten object at 0x7131ac51f9a0>
layer: 10 <keras.src.layers.regularization.dropout.Dropout object at 0x7131ac51fd60>
layer: 11 <keras.src.layers.core.dense.Dense object at 0x7131ac51fa60>
layer: 12 <keras.src.layers.core.dense.Dense object at 0x7131ac51f700>
layer: 13 <keras.src.laye

In [20]:
x = model.layers[-2].output
predictions = Dense(2, activation="sigmoid", name="last")(x)
model_final = Model(model.input, predictions)

model_final.compile(
    optimizer=Adam(learning_rate=0.001, use_ema=True, ema_momentum=1),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

In [21]:
img_h = 224
img_w = 224
n_channels = 3

In [22]:
model_final.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_input (InputLayer)   [(None, 224, 224, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 110, 110, 128)     36992     
                                                                 
 max_pooling2d (MaxPooling2  (None, 54, 54, 128)       0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 256)       295168    
                                                                 
 conv2d_3 (Conv2D)           (None, 50, 50, 256)       590080    
                                                             

In [23]:
epochs = 1

In [24]:
log_dir = "./logs/fit" + datetime.datetime.now().strftime("%d%m%Y-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
callbacks_list = [tensorboard_callback]



model_final.fit(
    train_images,
    epochs=epochs,
    callbacks=callbacks_list,
    verbose=1
)




2024-08-16 01:40:04.250676: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-08-16 01:40:05.103458: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
2024-08-16 01:40:05.231738: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


  1/303 [..............................] - ETA: 21:45 - loss: 0.6891 - accuracy: 0.9375

2024-08-16 01:40:08.656821: I external/local_xla/xla/service/service.cc:168] XLA service 0x71303e20f070 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-08-16 01:40:08.656845: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2024-08-16 01:40:08.671292: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1723752608.756122    3188 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


303/303 [==============================] - 43s 128ms/step - loss: 0.4749 - accuracy: 0.9189


In [25]:
c = 0
for img in os.listdir(images_path):
    if img.startswith("4"):
        c+= 1
print("Number of images that starts with 4 : ", c)

Number of images that starts with 4 :  17


In [ ]:
threshold_to_remove_redundant_bb = 0.1

In [35]:
c = 0
for img in os.listdir(images_path):
    if img.startswith("4"):
        image = cv2.imread(os.path.join(images_path, img))
        image_bb_output = []
        # cv2.imshow("image", image)
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()
        ss.setBaseImage(image)
        ss.switchToSelectiveSearchFast()
        ssresults = ss.process()
        imout = image.copy()
        for bb in ssresults:
            x, y, w, h = bb
            im_to_model = imout[y:y+h, x:x+w]
            # print("shape: ", im_to_model.shape)
            im_to_m = cv2.resize(im_to_model, (img_w, img_h) , interpolation=cv2.INTER_AREA)
            # print("r shape: ", im_to_m.shape)
            d = np.expand_dims(im_to_m, axis=0)
            # print("shape of input to model: ", d.shape)
            res = model.predict(d)
            # print("res: ", res)
            predictions = np.argmax(res, axis=1)
            # print(predictions)
            if res[0][1] > res[0][0] and h*w < img_h*img_w*0.04:
                # cv2.rectangle(imout, (x, y), (x+w, y+h), (0, 0, 255), 1)
                image_bb_output.append([(x, y), (x+w, y+h)])
                c += 1
            # print("res: ", res[0][1])
            # cv2.imshow("image", im_to_model)
            # cv2.waitKey(0)
            # cv2.destroyAllWindows()
        g = False
        out_list = []
        ol = []
        o = []
        for i in range(len(image_bb_output)):
            for j in range(i+1, len(image_bb_output)):
                x = image_bb_output[j][0][0]
                y = image_bb_output[j][0][1]
                xn = image_bb_output[j][1][0]
                yn = image_bb_output[j][1][1]
                bb1 = {"x1":x, "y1":y, "x2":xn, "y2":yn}
                x = image_bb_output[i][0][0]
                y = image_bb_output[i][0][1]
                xn = image_bb_output[i][1][0]
                yn = image_bb_output[i][1][1]
                bb2 = {"x1":x, "y1":y, "x2":xn, "y2":yn}
                r = gt(bb1, bb2)
                if r < threshold_to_remove_redundant_bb:
                    if bb1 not in o and bb1 not in ol:
                        ol.append(bb1)
                    if bb2 not in o and bb2 not in ol:
                        ol.append(bb2)
                else:
                    if bb1 in ol:
                        ol.remove(bb1)
                    if bb2 in ol:
                        ol.remove(bb2)
                    ol.append(bb1)
                    o.append(bb2)
        
        for i in ol:
            cv2.rectangle(imout, (i["x1"], i["y1"]), (i["x2"], i["y2"]), (0, 0, 255), 1)
        cv2.imshow(str(len(ol)), imout)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        c = 0

1/1 [==============================] - 0s 19ms/step


In [ ]:
# and res[0][0] < 0.000001 and h*w < img_h*img_w*0.04